In [1]:
import censusdata as cd

In [2]:
#% pip install CensusData
import censusdata as cd
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_columns', None)

In [3]:
pd.read_csv('../data/02_demo_data/ACS/ACS_2019_income_by_county/ACSST5Y2019.S1901_metadata_2021-10-05T144610.csv')

,GEO_ID,id
0,NAME,Geographic Area Name
1,S1901_C01_001E,Estimate!!Households!!Total
2,S1901_C01_001M,Margin of Error!!Households!!Total
3,S1901_C01_002E,"Estimate!!Households!!Total!!Less than $10,000"
4,S1901_C01_002M,Margin of Error!!Households!!Total!!Less than ...
...,...,...
124,S1901_C04_014M,Margin of Error!!Nonfamily households!!PERCENT...
125,S1901_C04_015E,Estimate!!Nonfamily households!!PERCENT ALLOCA...
126,S1901_C04_015M,Margin of Error!!Nonfamily households!!PERCENT...
127,S1901_C04_016E,Estimate!!Nonfamily households!!PERCENT ALLOCA...


- First, read in the ACS 2019 income data columns of interest. We used the metadata descriptions from this file: `ACSST5Y2019.S1901_metadata_2021-10-05T144610.csv` to choose the desired columns.

- To keep things less busy, we created a columns_of_interest list for each .csv that we read in and used that during the `pd.read_csv()` process.

In [4]:
# Specify columns of interest.
acs2019_income_columns_of_interest = ['GEO_ID', 
                                      'NAME', 
                                      'S1901_C01_001E', 
                                      'S1901_C01_012E', 
                                      'S1901_C01_013E', 
                                      'S1901_C02_001E', 
                                      'S1901_C02_012E', 
                                      'S1901_C02_013E']

# Read in the data.
acs2019_income = pd.read_csv('../data/02_demo_data/ACS/ACS_2019_income_by_county/ACSST5Y2019.S1901_data_with_overlays_2021-10-05T144610.csv',
                                usecols = acs2019_income_columns_of_interest)
acs2019_income

,GEO_ID,NAME,S1901_C01_001E,S1901_C01_012E,S1901_C01_013E,S1901_C02_001E,S1901_C02_012E,S1901_C02_013E
0,id,Geographic Area Name,Estimate!!Households!!Total,Estimate!!Households!!Median income (dollars),Estimate!!Households!!Mean income (dollars),Estimate!!Families!!Total,Estimate!!Families!!Median income (dollars),Estimate!!Families!!Mean income (dollars)
1,0500000US01001,"Autauga County, Alabama",21397,58731,75326,15076,71103,87094
2,0500000US01003,"Baldwin County, Alabama",80930,58320,80986,53467,75850,97991
3,0500000US01005,"Barbour County, Alabama",9345,32525,47068,6187,41704,56374
4,0500000US01007,"Bibb County, Alabama",6891,47542,60182,4789,57891,69316
...,...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",18721,19617,28805,13216,24963,32859
3217,0500000US72147,"Vieques Municipio, Puerto Rico",2258,14936,22742,1203,22429,28085
3218,0500000US72149,"Villalba Municipio, Puerto Rico",7908,19877,29612,5873,23231,32959
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",11541,16295,24078,7576,20785,28567


In [5]:
# Clean up the naming conventions.
acs2019_income.columns = ['geo_id', 
                          'name',
                          'total_hh',
                          'median_hh_income',
                          'mean_hh_income',
                          'total_families',
                          'median_family_income',
                          'mean_family_income',
                          ]

In [6]:
acs2019_income

,geo_id,name,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income
0,id,Geographic Area Name,Estimate!!Households!!Total,Estimate!!Households!!Median income (dollars),Estimate!!Households!!Mean income (dollars),Estimate!!Families!!Total,Estimate!!Families!!Median income (dollars),Estimate!!Families!!Mean income (dollars)
1,0500000US01001,"Autauga County, Alabama",21397,58731,75326,15076,71103,87094
2,0500000US01003,"Baldwin County, Alabama",80930,58320,80986,53467,75850,97991
3,0500000US01005,"Barbour County, Alabama",9345,32525,47068,6187,41704,56374
4,0500000US01007,"Bibb County, Alabama",6891,47542,60182,4789,57891,69316
...,...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",18721,19617,28805,13216,24963,32859
3217,0500000US72147,"Vieques Municipio, Puerto Rico",2258,14936,22742,1203,22429,28085
3218,0500000US72149,"Villalba Municipio, Puerto Rico",7908,19877,29612,5873,23231,32959
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",11541,16295,24078,7576,20785,28567


#### Next we need to drop the first row of data, which had the informative column names, parse off the FIPS ID, and reset the index to the FIPS ID.

In [7]:
# Drop the first row.
acs2019_income.drop(index = 0, inplace = True)

In [8]:
# Parse out the state and county from `name`.
acs2019_income[['county', 'state']] = acs2019_income['name'].str.split(',', expand = True)

# Pick off the FIPS code from the end of the GEO_ID.
acs2019_income['FIPS'] = acs2019_income['geo_id'].str[-5:]

# Examine the dataset.
acs2019_income

,geo_id,name,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,county,state,FIPS
1,0500000US01001,"Autauga County, Alabama",21397,58731,75326,15076,71103,87094,Autauga County,Alabama,01001
2,0500000US01003,"Baldwin County, Alabama",80930,58320,80986,53467,75850,97991,Baldwin County,Alabama,01003
3,0500000US01005,"Barbour County, Alabama",9345,32525,47068,6187,41704,56374,Barbour County,Alabama,01005
4,0500000US01007,"Bibb County, Alabama",6891,47542,60182,4789,57891,69316,Bibb County,Alabama,01007
5,0500000US01009,"Blount County, Alabama",20847,49358,65639,14874,62295,76547,Blount County,Alabama,01009
...,...,...,...,...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",18721,19617,28805,13216,24963,32859,Vega Baja Municipio,Puerto Rico,72145
3217,0500000US72147,"Vieques Municipio, Puerto Rico",2258,14936,22742,1203,22429,28085,Vieques Municipio,Puerto Rico,72147
3218,0500000US72149,"Villalba Municipio, Puerto Rico",7908,19877,29612,5873,23231,32959,Villalba Municipio,Puerto Rico,72149
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",11541,16295,24078,7576,20785,28567,Yabucoa Municipio,Puerto Rico,72151


In [9]:
# Drop `geo_id`, `name`.
acs2019_income.drop(columns = ['geo_id', 'name'], inplace = True)
acs2019_income

,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,county,state,FIPS
1,21397,58731,75326,15076,71103,87094,Autauga County,Alabama,01001
2,80930,58320,80986,53467,75850,97991,Baldwin County,Alabama,01003
3,9345,32525,47068,6187,41704,56374,Barbour County,Alabama,01005
4,6891,47542,60182,4789,57891,69316,Bibb County,Alabama,01007
5,20847,49358,65639,14874,62295,76547,Blount County,Alabama,01009
...,...,...,...,...,...,...,...,...,...
3216,18721,19617,28805,13216,24963,32859,Vega Baja Municipio,Puerto Rico,72145
3217,2258,14936,22742,1203,22429,28085,Vieques Municipio,Puerto Rico,72147
3218,7908,19877,29612,5873,23231,32959,Villalba Municipio,Puerto Rico,72149
3219,11541,16295,24078,7576,20785,28567,Yabucoa Municipio,Puerto Rico,72151


In [35]:
# Set the index to the FIPS code. 

acs2019_income.set_index('FIPS', inplace = True)

In [ ]:
# Drop Puerto Rico.


In [ ]:
acs2019_income.drop()

In [52]:
acs2019_income[acs2019_income['state'].str.contains('Puerto')].drop(index)

KeyError: '[1] not found in axis'

In [ ]:
df[df['Behavior'].str.contains('nt|nv', na=False)]

In [ ]:
acs2019_income['state'].str[conta]

In [ ]:
df['A'] == 'foo'

In [44]:
pr = acs2019_income['state'] == 'Alabama'
acs2019_income[pr]

,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,county,state
FIPS,,,,,,,,


In [45]:
acs2019_income.loc[acs2019_income['state']
                   == 'Alabama']

,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,county,state
FIPS,,,,,,,,


In [31]:
acs2019_income.iloc[['state']=='Puerto Rico']

TypeError: Cannot index by location index with a non-integer key

In [21]:
acs2019_income[acs2019_income['state'] == 'Virginia']

,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,county,state
FIPS,,,,,,,,


In [5]:
county_complete_all = pd.read_csv('../data/02_demo_data/openintro_dot_org/county_complete.csv')
county_complete = county_complete_all[['fips', 
                           'state', 
                           'name', 
                           'smoking_ban_2010',                                                        
                           'asian_2019', 
                           'avg_family_size_2019', 
                           'black_2019',
                           'hispanic_2019',
                           'household_has_broadband_2019', 
                           'household_has_computer_2019',
                           'household_has_smartphone_2019',
                           'households_2019',
                           'households_speak_limited_english_2019',
                           'housing_mobile_homes_2019',
                           'hs_grad_2019',                           
                           'median_household_income_2019',
                           'median_individual_income_2019',
                           'native_2019',
                           'other_single_race_2019',
                           'pac_isl_2019',
                           'persons_per_household_2019',
                           'pop_2019',
                           'unemployment_rate_2019',
                           'uninsured_2019',
                           'veterans_2019',
                           'white_2019',
                           'white_not_hispanic_2019']]
county_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   fips                                   3142 non-null   int64  
 1   state                                  3142 non-null   object 
 2   name                                   3142 non-null   object 
 3   smoking_ban_2010                       3116 non-null   object 
 4   asian_2019                             3142 non-null   float64
 5   avg_family_size_2019                   3142 non-null   float64
 6   black_2019                             3142 non-null   float64
 7   hispanic_2019                          3142 non-null   float64
 8   household_has_broadband_2019           3142 non-null   float64
 9   household_has_computer_2019            3142 non-null   float64
 10  household_has_smartphone_2019          3142 non-null   float64
 11  hous

In [6]:
county_class_all= pd.read_csv('../data/02_demo_data/rural_atlas_all_counties/County Classifications.csv', 
                              delimiter='\t', 
                              encoding_errors = 'Ignore',
                              converters = {'FIPStxt': lambda x: str(x)})
county_class_all.head()

# Note: I added the encoding_errors = 'Ignore' as a test after getting this error:
# 'utf-8' codec can't decode byte 0xf1 in position 185518: invalid continuation byte
# This was on the second pass of importing the file; the first pass yielded no errors.
# Got the idea for the converters from:
# https://stackoverflow.com/questions/13250046/how-to-keep-leading-zeros-in-a-column-when-reading-csv-with-pandas

,FIPStxt,State,County,RuralUrbanContinuumCode2013,UrbanInfluenceCode2013,RuralUrbanContinuumCode2003,UrbanInfluenceCode2003,Metro2013,Nonmetro2013,Micropolitan2013,Type_2015_Update,Type_2015_Farming_NO,Type_2015_Manufacturing_NO,Type_2015_Mining_NO,Type_2015_Government_NO,Type_2015_Recreation_NO,Low_Education_2015_update,Low_Employment_2015_update,Population_loss_2015_update,Retirement_Destination_2015_Update,Perpov_1980_0711,PersistentChildPoverty_1980_2011,Hipov,HiAmenity,HiCreativeClass2000,Gas_Change,Oil_Change,Oil_Gas_Change,Metro2003,NonmetroNotAdj2003,NonmetroAdj2003,Noncore2003,EconomicDependence2000,Nonmetro2003,Micropolitan2003,FarmDependent2003,ManufacturingDependent2000,LowEducation2000,RetirementDestination2000,PersistentPoverty2000,Noncore2013,Type_2015_Nonspecialized_NO,Metro_Adjacent2013,PersistentChildPoverty2004,RecreationDependent2000
0,01001,AL,Autauga,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,01003,AL,Baldwin,3.0,2.0,4.0,5.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,9.0,0.0,9.0,0.0,0.0,1.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,01005,AL,Barbour,6.0,6.0,6.0,6.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
3,01007,AL,Bibb,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,01009,AL,Blount,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
county_class = county_class_all[['FIPStxt', 
                             'State', 
                             'County', 
                             'RuralUrbanContinuumCode2013', 
                             'Retirement_Destination_2015_Update', 
                             'Metro_Adjacent2013']]
county_class

,FIPStxt,State,County,RuralUrbanContinuumCode2013,Retirement_Destination_2015_Update,Metro_Adjacent2013
0,01001,AL,Autauga,2.0,1.0,0.0
1,01003,AL,Baldwin,3.0,1.0,0.0
2,01005,AL,Barbour,6.0,0.0,1.0
3,01007,AL,Bibb,1.0,0.0,0.0
4,01009,AL,Blount,1.0,0.0,0.0
...,...,...,...,...,...,...
3220,72145,PR,Vega Baja,1.0,NaN,0.0
3221,72147,PR,Vieques,7.0,NaN,0.0
3222,72149,PR,Villalba,2.0,NaN,0.0
3223,72151,PR,Yabucoa,1.0,NaN,0.0


In [8]:
county_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225 entries, 0 to 3224
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   FIPStxt                             3225 non-null   object 
 1   State                               3225 non-null   object 
 2   County                              3225 non-null   object 
 3   RuralUrbanContinuumCode2013         3221 non-null   float64
 4   Retirement_Destination_2015_Update  3143 non-null   float64
 5   Metro_Adjacent2013                  3221 non-null   float64
dtypes: float64(3), object(3)
memory usage: 151.3+ KB


In [16]:
acs2019_poverty_all = pd.read_csv('../data/02_demo_data/ACS/ACS_2019_poverty_by_county/ACSST5Y2019.S1701_data_with_overlays_2021-11-02T160432.csv')
acs2019_poverty_all.tail()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,2

,GEO_ID,NAME,S1701_C01_001E,S1701_C01_001M,S1701_C01_002E,S1701_C01_002M,S1701_C01_003E,S1701_C01_003M,S1701_C01_004E,S1701_C01_004M,S1701_C01_005E,S1701_C01_005M,S1701_C01_006E,S1701_C01_006M,S1701_C01_007E,S1701_C01_007M,S1701_C01_008E,S1701_C01_008M,S1701_C01_009E,S1701_C01_009M,S1701_C01_010E,S1701_C01_010M,S1701_C01_011E,S1701_C01_011M,S1701_C01_012E,S1701_C01_012M,S1701_C01_013E,S1701_C01_013M,S1701_C01_014E,S1701_C01_014M,S1701_C01_015E,S1701_C01_015M,S1701_C01_016E,S1701_C01_016M,S1701_C01_017E,S1701_C01_017M,S1701_C01_018E,S1701_C01_018M,S1701_C01_019E,S1701_C01_019M,S1701_C01_020E,S1701_C01_020M,S1701_C01_021E,S1701_C01_021M,S1701_C01_022E,S1701_C01_022M,S1701_C01_023E,S1701_C01_023M,S1701_C01_024E,S1701_C01_024M,S1701_C01_025E,S1701_C01_025M,S1701_C01_026E,S1701_C01_026M,S1701_C01_027E,S1701_C01_027M,S1701_C01_028E,S1701_C01_028M,S1701_C01_029E,S1701_C01_029M,S1701_C01_030E,S1701_C01_030M,S1701_C01_031E,S1701_C01_031M,S1701_C01_032E,S1701_C01_032M,S1701_C01_033E,S1701_C01_033M,S1701_C01_034E,S1701_C01_034M,S1701_C01_035E,S1701_C01_035M,S1701_C01_036E,S1701_C01_036M,S1701_C01_037E,S1701_C01_037M,S1701_C01_038E,S1701_C01_038M,S1701_C01_039E,S1701_C01_039M,S1701_C01_040E,S1701_C01_040M,S1701_C01_041E,S1701_C01_041M,S1701_C01_042E,S1701_C01_042M,S1701_C01_043E,S1701_C01_043M,S1701_C01_044E,S1701_C01_044M,S1701_C01_045E,S1701_C01_045M,S1701_C01_046E,S1701_C01_046M,S1701_C01_047E,S1701_C01_047M,S1701_C01_048E,S1701_C01_048M,S1701_C01_049E,S1701_C01_049M,S1701_C01_050E,S1701_C01_050M,S1701_C01_051E,S1701_C01_051M,S1701_C01_052E,S1701_C01_052M,S1701_C01_053E,S1701_C01_053M,S1701_C01_054E,S1701_C01_054M,S1701_C01_055E,S1701_C01_055M,S1701_C01_056E,S1701_C01_056M,S1701_C01_057E,S1701_C01_057M,S1701_C01_058E,S1701_C01_058M,S1701_C01_059E,S1701_C01_059M,S1701_C01_060E,S1701_C01_060M,S1701_C01_061E,S1701_C01_061M,S1701_C02_001E,S1701_C02_001M,S1701_C02_002E,S1701_C02_002M,S1701_C02_003E,S1701_C02_003M,S1701_C02_004E,S1701_C02_004M,S1701_C02_005E,S1701_C02_005M,S1701_C02_006E,S1701_C02_006M,S1701_C02_007E,S1701_C02_007M,S1701_C02_008E,S1701_C02_008M,S1701_C02_009E,S1701_C02_009M,S1701_C02_010E,S1701_C02_010M,S1701_C02_011E,S1701_C02_011M,S1701_C02_012E,S1701_C02_012M,S1701_C02_013E,S1701_C02_013M,S1701_C02_014E,S1701_C02_014M,S1701_C02_015E,S1701_C02_015M,S1701_C02_016E,S1701_C02_016M,S1701_C02_017E,S1701_C02_017M,S1701_C02_018E,S1701_C02_018M,S1701_C02_019E,S1701_C02_019M,S1701_C02_020E,S1701_C02_020M,S1701_C02_021E,S1701_C02_021M,S1701_C02_022E,S1701_C02_022M,S1701_C02_023E,S1701_C02_023M,S1701_C02_024E,S1701_C02_024M,S1701_C02_025E,S1701_C02_025M,S1701_C02_026E,S1701_C02_026M,S1701_C02_027E,S1701_C02_027M,S1701_C02_028E,S1701_C02_028M,S1701_C02_029E,S1701_C02_029M,S1701_C02_030E,S1701_C02_030M,S1701_C02_031E,S1701_C02_031M,S1701_C02_032E,S1701_C02_032M,S1701_C02_033E,S1701_C02_033M,S1701_C02_034E,S1701_C02_034M,S1701_C02_035E,S1701_C02_035M,S1701_C02_036E,S1701_C02_036M,S1701_C02_037E,S1701_C02_037M,S1701_C02_038E,S1701_C02_038M,S1701_C02_039E,S1701_C02_039M,S1701_C02_040E,S1701_C02_040M,S1701_C02_041E,S1701_C02_041M,S1701_C02_042E,S1701_C02_042M,S1701_C02_043E,S1701_C02_043M,S1701_C02_044E,S1701_C02_044M,S1701_C02_045E,S1701_C02_045M,S1701_C02_046E,S1701_C02_046M,S1701_C02_047E,S1701_C02_047M,S1701_C02_048E,S1701_C02_048M,S1701_C02_049E,S1701_C02_049M,S1701_C02_050E,S1701_C02_050M,S1701_C02_051E,S1701_C02_051M,S1701_C02_052E,S1701_C02_052M,S1701_C02_053E,S1701_C02_053M,S1701_C02_054E,S1701_C02_054M,S1701_C02_055E,S1701_C02_055M,S1701_C02_056E,S1701_C02_056M,S1701_C02_057E,S1701_C02_057M,S1701_C02_058E,S1701_C02_058M,S1701_C02_059E,S1701_C02_059M,S1701_C02_060E,S1701_C02_060M,S1701_C02_061E,S1701_C02_061M,S1701_C03_001E,S1701_C03_001M,S1701_C03_002E,S1701_C03_002M,S1701_C03_003E,S1701_C03_003M,S1701_C03_004E,S1701_C03_004M,S1701_C03_005E,S1701_C03_005M,S1701_C03_006E,S1701_C03_006M,S1701_C03_007E,S1701_C03_007M,S1701_C03_008E,S1701_C03_008M,S1701_C03_009E,S1701_C03_009M,S1701_C03_010E,S1701_C03_010M,S1701_C03_011E,S1701_C03_01

In [10]:
acs2019_poverty = acs2019_poverty_all[['GEO_ID', 'NAME', 'S1701_C01_001E', 'S1701_C02_001E', 'S1701_C03_001E']]
acs2019_poverty.head()


,GEO_ID,NAME,S1701_C01_001E,S1701_C02_001E,S1701_C03_001E
0,id,Geographic Area Name,Estimate!!Total!!Population for whom poverty s...,Estimate!!Below poverty level!!Population for ...,Estimate!!Percent below poverty level!!Populat...
1,0500000US01001,"Autauga County, Alabama",54922,8340,15.2
2,0500000US01003,"Baldwin County, Alabama",209618,21704,10.4
3,0500000US01005,"Barbour County, Alabama",22417,6875,30.7
4,0500000US01007,"Bibb County, Alabama",20632,3740,18.1


In [15]:
acs2019_poverty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   GEO_ID          3221 non-null   object
 1   NAME            3221 non-null   object
 2   S1701_C01_001E  3221 non-null   object
 3   S1701_C02_001E  3221 non-null   object
 4   S1701_C03_001E  3221 non-null   object
dtypes: object(5)
memory usage: 125.9+ KB


In [27]:
acs2019_poverty['FIPS'] = acs2019_poverty['GEO_ID'].str[-5:]

In [28]:
acs2019_poverty[['GEO_ID', 'FIPS']]

,GEO_ID,FIPS
0,id,id
1,0500000US01001,01001
2,0500000US01003,01003
3,0500000US01005,01005
4,0500000US01007,01007
...,...,...
3216,0500000US72145,72145
3217,0500000US72147,72147
3218,0500000US72149,72149
3219,0500000US72151,72151


In [11]:
%who DataFrame

acs2019_income	 acs2019_income_all	 acs2019_poverty	 acs2019_poverty_all	 county_class	 county_class_all	 county_complete	 county_complete_all	 
